In [11]:
import os
import pandas as pd
import numpy as np
import re
from datetime import date, datetime, timedelta
import warnings
import glob 
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Desactiva las advertencias
warnings.filterwarnings('ignore')

In [12]:
# Definir las columnas de códigos y de diagnósticos en texto libre
columnas_codigos = ['COD_DIAGNOSTICO1', 'COD_DIAGNOSTICO2', 'COD_DIAGNOSTICO3',
                    'COD_DIAGNOSTICO4', 'COD_DIAGNOSTICO5', 'COD_DIAGNOSTICO6',
                    'COD_DIAGNOSTICO7', 'COD_DIAGNOSTICO8', 'COD_DIAGNOSTICO9',
                    'COD_DIAGNOSTICO10']
columnas_diagnosticos = ['DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3',
                         'DIAGNOSTICO4', 'DIAGNOSTICO5', 'DIAGNOSTICO6',
                         'DIAGNOSTICO7', 'DIAGNOSTICO8', 'DIAGNOSTICO9',
                         'DIAGNOSTICO10']


In [13]:
def cargar_dataframe_ambulatorio(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')

def cargar_dataframe_ieeh(ruta):
    encoding = 'ISO-8859-1'
    delimitador = ';'
    return pd.read_csv(ruta, delimiter=delimitador, encoding=encoding, on_bad_lines='skip')

def filtro_dataframe_ca_tiroides(dataframe):

    # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer colorrectal
    patron_cancer_colorrectal = r'^C18.*|C19.*|C20.*'
    
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer colorrectal
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_colorrectal, na=False)).any(axis=1)

   # Patrones para identificar diagnósticos de cáncer colorrectal en texto libre
    patron_diagnostico_colorrectal = re.compile(r'\b(?:c[aáx[^mn]]*[ \.]*col[oa]n|\bc[aáx[^mn]]*[ \.]*rect[oa])\b', re.IGNORECASE)


    patron_adenocarcinoma_colorrectal = re.compile(r'\badenocarcinoma.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_carcinoma_escamoso_colorrectal = re.compile(r'\bcarcinoma.*escamoso.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno_colon = re.compile(r'\btumor.*maligno.*colon', re.IGNORECASE)
    patron_tumor_maligno_recto = re.compile(r'\btumor.*maligno.*recto', re.IGNORECASE)
    patron_neoplasia_maligna_colon = re.compile(r'\bneoplasia.*maligna.*colon', re.IGNORECASE)
    patron_neoplasia_maligna_recto = re.compile(r'\bneoplasia.*maligna.*recto', re.IGNORECASE)
    patron_cancer_colorrectal_metastasico = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*metast[a-zá-ú]*', re.IGNORECASE)
    patron_cancer_colon_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*colon.*avanzado', re.IGNORECASE)
    patron_cancer_recto_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*avanzado', re.IGNORECASE)


    # Términos acompañados por otros
    patron_neoplasia_ubicaciones = re.compile(r'\bneoplasia.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_adenoca_ubicaciones = re.compile(r'\badenoca.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_gist_ubicaciones = re.compile(r'\bgist.*(?:colon\b|recto\b|rectal|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_cancer_ubicaciones = re.compile(r'\bca[aáx[^mn](?:ncer)?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_tumor_ubicaciones = re.compile(r'\btu(?:mor)?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer colorrectal en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_colorrectal, na=False) |
        x.str.contains(patron_adenocarcinoma_colorrectal, na=False) |
        x.str.contains(patron_carcinoma_escamoso_colorrectal, na=False) |
        x.str.contains(patron_tumor_maligno_colon, na=False) |
        x.str.contains(patron_tumor_maligno_recto, na=False) |
        x.str.contains(patron_neoplasia_maligna_colon, na=False) |
        x.str.contains(patron_neoplasia_maligna_recto, na=False) |
        x.str.contains(patron_cancer_colorrectal_metastasico, na=False) |
        x.str.contains(patron_cancer_colon_avanzado, na=False) |
        x.str.contains(patron_cancer_recto_avanzado, na=False) |
        x.str.contains(patron_neoplasia_ubicaciones, na=False) |
        x.str.contains(patron_adenoca_ubicaciones, na=False) |
        x.str.contains(patron_gist_ubicaciones, na=False) |
        x.str.contains(patron_cancer_ubicaciones, na=False) |
        x.str.contains(patron_tumor_ubicaciones, na=False)).any(axis=1)

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    return dataframe[filtro_codigos | filtro_diagnosticos].copy()


def filtro_dataframe_ca_tiroides_ieeh(dataframe):

    # Definir las columnas de códigos y de diagnósticos en texto libre

    columnas_diagnosticos = ['DIAG1','DIAG2','DIAG3']
    # Llenar los valores NaN con una cadena vacía en las columnas de diagnóstico
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].fillna('')


    # Patrones para identificar diagnósticos de cáncer colorrectal en texto libre
    patron_diagnostico_colorrectal = re.compile(r'\b(?:c[aáx[^mn]]*[ \.]*col[oa]n|\bc[aáx[^mn]]*[ \.]*rect[oa])\b', re.IGNORECASE)


    patron_adenocarcinoma_colorrectal = re.compile(r'\badenocarcinoma.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_carcinoma_escamoso_colorrectal = re.compile(r'\bcarcinoma.*escamoso.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno_colon = re.compile(r'\btumor.*maligno.*colon', re.IGNORECASE)
    patron_tumor_maligno_recto = re.compile(r'\btumor.*maligno.*recto', re.IGNORECASE)
    patron_neoplasia_maligna_colon = re.compile(r'\bneoplasia.*maligna.*colon', re.IGNORECASE)
    patron_neoplasia_maligna_recto = re.compile(r'\bneoplasia.*maligna.*recto', re.IGNORECASE)
    patron_cancer_colorrectal_metastasico = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*metast[a-zá-ú]*', re.IGNORECASE)
    patron_cancer_colon_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*colon.*avanzado', re.IGNORECASE)
    patron_cancer_recto_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*avanzado', re.IGNORECASE)


    # Términos acompañados por otros
    patron_neoplasia_ubicaciones = re.compile(r'\bneoplasia.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_adenoca_ubicaciones = re.compile(r'\badenoca.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_gist_ubicaciones = re.compile(r'\bgist.*(?:colon\b|recto\b|rectal|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_cancer_ubicaciones = re.compile(r'\bca[aáx[^mn](?:ncer)?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_tumor_ubicaciones = re.compile(r'\btu(?:mor)?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer colorrectal en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_colorrectal, na=False) |
        x.str.contains(patron_adenocarcinoma_colorrectal, na=False) |
        x.str.contains(patron_carcinoma_escamoso_colorrectal, na=False) |
        x.str.contains(patron_tumor_maligno_colon, na=False) |
        x.str.contains(patron_tumor_maligno_recto, na=False) |
        x.str.contains(patron_neoplasia_maligna_colon, na=False) |
        x.str.contains(patron_neoplasia_maligna_recto, na=False) |
        x.str.contains(patron_cancer_colorrectal_metastasico, na=False) |
        x.str.contains(patron_cancer_colon_avanzado, na=False) |
        x.str.contains(patron_cancer_recto_avanzado, na=False) |
        x.str.contains(patron_neoplasia_ubicaciones, na=False) |
        x.str.contains(patron_adenoca_ubicaciones, na=False) |
        x.str.contains(patron_gist_ubicaciones, na=False) |
        x.str.contains(patron_cancer_ubicaciones, na=False) |
        x.str.contains(patron_tumor_ubicaciones, na=False)).any(axis=1)


    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_tiroides_urgencias(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['CODDIAGNOSTICO']
    columnas_diagnosticos = ['DESCDIAGNOSTICO']

    # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer colorrectal
    patron_cancer_colorrectal = r'^C18.*|C19.*|C20.*'
    
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer colorrectal
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_colorrectal, na=False)).any(axis=1)

    patron_diagnostico_colorrectal = re.compile(r'\b(?:c[aáx[^mn]]*[ \.]*col[oa]n|\bc[aáx[^mn]]*[ \.]*rect[oa])\b', re.IGNORECASE)


    patron_adenocarcinoma_colorrectal = re.compile(r'\badenocarcinoma.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_carcinoma_escamoso_colorrectal = re.compile(r'\bcarcinoma.*escamoso.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno_colon = re.compile(r'\btumor.*maligno.*colon', re.IGNORECASE)
    patron_tumor_maligno_recto = re.compile(r'\btumor.*maligno.*recto', re.IGNORECASE)
    patron_neoplasia_maligna_colon = re.compile(r'\bneoplasia.*maligna.*colon', re.IGNORECASE)
    patron_neoplasia_maligna_recto = re.compile(r'\bneoplasia.*maligna.*recto', re.IGNORECASE)
    patron_cancer_colorrectal_metastasico = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*metast[a-zá-ú]*', re.IGNORECASE)
    patron_cancer_colon_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*colon.*avanzado', re.IGNORECASE)
    patron_cancer_recto_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*avanzado', re.IGNORECASE)


    # Términos acompañados por otros
    patron_neoplasia_ubicaciones = re.compile(r'\bneoplasia.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_adenoca_ubicaciones = re.compile(r'\badenoca.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_gist_ubicaciones = re.compile(r'\bgist.*(?:colon\b|recto\b|rectal|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_cancer_ubicaciones = re.compile(r'\bca[aáx[^mn](?:ncer)?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_tumor_ubicaciones = re.compile(r'\btu(?:mor)?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer colorrectal en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_colorrectal, na=False) |
        x.str.contains(patron_adenocarcinoma_colorrectal, na=False) |
        x.str.contains(patron_carcinoma_escamoso_colorrectal, na=False) |
        x.str.contains(patron_tumor_maligno_colon, na=False) |
        x.str.contains(patron_tumor_maligno_recto, na=False) |
        x.str.contains(patron_neoplasia_maligna_colon, na=False) |
        x.str.contains(patron_neoplasia_maligna_recto, na=False) |
        x.str.contains(patron_cancer_colorrectal_metastasico, na=False) |
        x.str.contains(patron_cancer_colon_avanzado, na=False) |
        x.str.contains(patron_cancer_recto_avanzado, na=False) |
        x.str.contains(patron_neoplasia_ubicaciones, na=False) |
        x.str.contains(patron_adenoca_ubicaciones, na=False) |
        x.str.contains(patron_gist_ubicaciones, na=False) |
        x.str.contains(patron_cancer_ubicaciones, na=False) |
        x.str.contains(patron_tumor_ubicaciones, na=False)).any(axis=1)

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[filtro_codigos | filtro_diagnosticos].copy()
    return resultado_filtrado

def filtro_dataframe_ca_tiroides_inter(dataframe):
    # Definir las columnas de códigos y de diagnósticos en texto libre
    columnas_codigos = ['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']
    columnas_diagnosticos = ['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']

    # Se crea un patrón regex para identificar los códigos CIE-10 asociados al cáncer colorrectal
    patron_cancer_colorrectal = r'^C18\.|C19\.|C20\.'
    
    # Filtrar el DataFrame para encontrar las filas que contienen el código de cáncer colorrectal
    filtro_codigos = dataframe[columnas_codigos].apply(lambda x: x.str.contains(patron_cancer_colorrectal, na=False)).any(axis=1)

    # Patrones para identificar diagnósticos de cáncer colorrectal en texto libre
    dataframe[columnas_codigos] = dataframe[columnas_codigos].astype(str)
    dataframe[columnas_diagnosticos] = dataframe[columnas_diagnosticos].astype(str)
    patron_diagnostico_colorrectal = re.compile(r'\b(?:c[aáx[^mn]]*[ \.]*col[oa]n|\bc[aáx[^mn]]*[ \.]*rect[oa])\b', re.IGNORECASE)


    patron_adenocarcinoma_colorrectal = re.compile(r'\badenocarcinoma.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_carcinoma_escamoso_colorrectal = re.compile(r'\bcarcinoma.*escamoso.*rect[a-zá-ú]*', re.IGNORECASE)
    patron_tumor_maligno_colon = re.compile(r'\btumor.*maligno.*colon', re.IGNORECASE)
    patron_tumor_maligno_recto = re.compile(r'\btumor.*maligno.*recto', re.IGNORECASE)
    patron_neoplasia_maligna_colon = re.compile(r'\bneoplasia.*maligna.*colon', re.IGNORECASE)
    patron_neoplasia_maligna_recto = re.compile(r'\bneoplasia.*maligna.*recto', re.IGNORECASE)
    patron_cancer_colorrectal_metastasico = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*metast[a-zá-ú]*', re.IGNORECASE)
    patron_cancer_colon_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*colon.*avanzado', re.IGNORECASE)
    patron_cancer_recto_avanzado = re.compile(r'\bc[a-zá-ú]*[ \.]*rect[a-zá-ú]*.*avanzado', re.IGNORECASE)


    # Términos acompañados por otros
    patron_neoplasia_ubicaciones = re.compile(r'\bneoplasia.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_adenoca_ubicaciones = re.compile(r'\badenoca.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_gist_ubicaciones = re.compile(r'\bgist.*(?:colon\b|recto\b|rectal|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_cancer_ubicaciones = re.compile(r'\bca[aáx[^mn](?:ncer)?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)
    patron_tumor_ubicaciones = re.compile(r'\btu(?:mor)?.*(?:colon\b|recto\b|rectal\b|transverso\b|sigmoides\b|apéndice\b|apendicular\b|neuroendocrino\b|canal anal\b|conducto anal\b)', re.IGNORECASE)

    # Filtrar el DataFrame para encontrar las filas que contienen el diagnóstico de cáncer colorrectal en texto libre
    filtro_diagnosticos = dataframe[columnas_diagnosticos].apply(lambda x:
        x.str.contains(patron_diagnostico_colorrectal, na=False) |
        x.str.contains(patron_adenocarcinoma_colorrectal, na=False) |
        x.str.contains(patron_carcinoma_escamoso_colorrectal, na=False) |
        x.str.contains(patron_tumor_maligno_colon, na=False) |
        x.str.contains(patron_tumor_maligno_recto, na=False) |
        x.str.contains(patron_neoplasia_maligna_colon, na=False) |
        x.str.contains(patron_neoplasia_maligna_recto, na=False) |
        x.str.contains(patron_cancer_colorrectal_metastasico, na=False) |
        x.str.contains(patron_cancer_colon_avanzado, na=False) |
        x.str.contains(patron_cancer_recto_avanzado, na=False) |
        x.str.contains(patron_neoplasia_ubicaciones, na=False) |
        x.str.contains(patron_adenoca_ubicaciones, na=False) |
        x.str.contains(patron_gist_ubicaciones, na=False) |
        x.str.contains(patron_cancer_ubicaciones, na=False) |
        x.str.contains(patron_tumor_ubicaciones, na=False)).any(axis=1)

    # Filtrar por la comuna La Florida
    filtro_comuna = dataframe['COMUNA'] == 'La Florida'

    # Combinar los filtros para obtener las filas que cumplen con al menos uno de los criterios
    resultado_filtrado = dataframe[(filtro_codigos | filtro_diagnosticos) & filtro_comuna].copy()
    return resultado_filtrado

def agregar_dv(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    #condicion = (resultado_filtrado['TIPO_IDENTIFICADOR_PACIENTE'] == 'RUN')
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[-1]

    #resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado.loc[condicion, 'NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
    resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'] = resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]

    return resultado_filtrado

def agregar_dv_urgencias(resultado_filtrado):

    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE
    resultado_filtrado['DV'] = resultado_filtrado['RUT'].str[-1]

    resultado_filtrado['RUT'] = resultado_filtrado['RUT'].str[:-1]

    return resultado_filtrado

def agregar_dv_inter(resultado_filtrado):
    # Verificar si el tipo de identificador es "RUN" y quitar el último dígito de NRO_IDENTIFICADOR_PACIENTE
    # Crear una nueva columna 'DV' que contenga el último dígito de NRO_IDENTIFICADOR_PACIENTE

    resultado_filtrado['DV'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[-1]

    resultado_filtrado['NRO_IDENTIFICADOR'] = resultado_filtrado['NRO_IDENTIFICADOR'].str[:-1]

    return resultado_filtrado

def obtener_diagnosticos_y_codigos(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR_PACIENTE'] == nro_identificacion]
    diagnosticos = df_paciente[['DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3', 'DIAGNOSTICO4',
                                'DIAGNOSTICO5', 'DIAGNOSTICO6', 'DIAGNOSTICO7','DIAGNOSTICO8',
                                'DIAGNOSTICO9','DIAGNOSTICO10']].values.flatten()
    codigos_diagnostico = df_paciente[['COD_DIAGNOSTICO1', 'COD_DIAGNOSTICO2', 'COD_DIAGNOSTICO3', 'COD_DIAGNOSTICO4',
                                       'COD_DIAGNOSTICO5', 'COD_DIAGNOSTICO6', 'COD_DIAGNOSTICO7','COD_DIAGNOSTICO8',
                                       'COD_DIAGNOSTICO9','COD_DIAGNOSTICO10']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def obtener_diagnosticos_ieeh(df, nro_identificacion):
    df_paciente = df[df['RUT'] == nro_identificacion]
    diagnosticos = df_paciente[['DIAG1','DIAG2','DIAG3']].values.flatten()
    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]

    return list(set(diagnosticos))

def obtener_diagnosticos_urgencias(df, nro_identificacion):
    df_paciente = df[df['RUT'] == nro_identificacion]
    diagnosticos = df_paciente[['DESCDIAGNOSTICO']].values.flatten()
    codigos_diagnostico = df_paciente[['CODDIAGNOSTICO']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def obtener_diagnosticos_inter(df, nro_identificacion):
    df_paciente = df[df['NRO_IDENTIFICADOR'] == nro_identificacion]
    diagnosticos = df_paciente[['SOSP_DIAG_DESC_1', 'SOSP_DIAG_DESC_2', 'SOSP_DIAG_DESC_3', 'SOSP_DIAG_DESC_4', 'SOSP_DIAG_DESC_5']].values.flatten()
    codigos_diagnostico = df_paciente[['SOSP_DIAG_COD_1', 'SOSP_DIAG_COD_2', 'SOSP_DIAG_COD_3', 'SOSP_DIAG_COD_4', 'SOSP_DIAG_COD_5']].values.flatten()

    diagnosticos = [diagnostico for diagnostico in diagnosticos if pd.notnull(diagnostico)]
    codigos_diagnostico = [codigo for codigo in codigos_diagnostico if pd.notnull(codigo)]

    return list(set(diagnosticos)), list(set(codigos_diagnostico))

def agregar_diagnosticos_y_codigos(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('NRO_IDENTIFICADOR_PACIENTE').agg({'NRO_IDENTIFICADOR_PACIENTE': 'first', 'DIAGNOSTICO1': 'first',
                                                            'DIAGNOSTICO2': 'first','DIAGNOSTICO3': 'first','DIAGNOSTICO4': 'first',
                                                            'DIAGNOSTICO5': 'first','DIAGNOSTICO6': 'first','DIAGNOSTICO7': 'first',
                                                            'DIAGNOSTICO8': 'first','DIAGNOSTICO9': 'first','DIAGNOSTICO10': 'first',
                                                            'COD_DIAGNOSTICO1': 'first','COD_DIAGNOSTICO2': 'first', 'COD_DIAGNOSTICO2': 'first',
                                                            'COD_DIAGNOSTICO3': 'first', 'COD_DIAGNOSTICO4': 'first', 'COD_DIAGNOSTICO5': 'first',
                                                            'COD_DIAGNOSTICO6': 'first', 'COD_DIAGNOSTICO7': 'first','COD_DIAGNOSTICO8': 'first',
                                                                'COD_DIAGNOSTICO8': 'first',  'COD_DIAGNOSTICO9': 'first',
                                                                'COD_DIAGNOSTICO10': 'first'}).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['NRO_IDENTIFICADOR_PACIENTE'].apply(lambda x: obtener_diagnosticos_y_codigos(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'NRO_IDENTIFICADOR_PACIENTE': 'RUT'})
    return resultados

def agregar_diagnosticos_ieeh(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT', 'DESCDIAGNOSTICO', 'CODDIAGNOSTICO', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('RUT').agg({'RUT': 'first', 'DIAG1': 'first',
                                                            'DIAG1': 'first','DIAG3': 'first'}).reset_index(drop=True)
    #resultados['CODIGOS_DIAGNOSTICO'] = []
    resultados['DIAGNOSTICOS'] = resultados['RUT'].apply(lambda x: obtener_diagnosticos_ieeh(resultado_filtrado, x))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'RUT': 'RUT'})
    resultados['CODIGOS_DIAGNOSTICO'] = np.nan
    return resultados

def agregar_diagnosticos_y_codigos_urgencias(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT', 'DESCDIAGNOSTICO', 'CODDIAGNOSTICO', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    resultados = resultado_filtrado.groupby('RUT').agg({'RUT': 'first', 'DESCDIAGNOSTICO': 'first',
                                                        'CODDIAGNOSTICO': 'first'}).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['RUT'].apply(lambda x: obtener_diagnosticos_urgencias(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'RUT': 'RUT'})

    return resultados

def agregar_diagnosticos_y_codigos_inter(resultado_filtrado):
    if resultado_filtrado.empty:
        return pd.DataFrame(columns=['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO'])

    # Aplicar la función de agregación en el DataFrame
    resultados = resultado_filtrado.groupby('NRO_IDENTIFICADOR').agg({'NRO_IDENTIFICADOR': 'first', 'SOSP_DIAG_DESC_1': 'first',
                                                            'SOSP_DIAG_DESC_2': 'first','SOSP_DIAG_DESC_3': 'first','SOSP_DIAG_DESC_4': 'first',
                                                            'SOSP_DIAG_DESC_5': 'first','SOSP_DIAG_COD_1': 'first','SOSP_DIAG_COD_2': 'first',
                                                                      'SOSP_DIAG_COD_3': 'first',
                                                            'SOSP_DIAG_COD_4': 'first', 'SOSP_DIAG_COD_5': 'first'
                                                            }).reset_index(drop=True)
    resultados['DIAGNOSTICOS'], resultados['CODIGOS_DIAGNOSTICO'] = zip(*resultados['NRO_IDENTIFICADOR'].apply(lambda x: obtener_diagnosticos_inter(resultado_filtrado, x)))
    # Renombrar la columna 'old_name' a 'new_name'
    resultados = resultados.rename(columns={'NRO_IDENTIFICADOR': 'RUT'})
    return resultados

def crear_dataframe_ambulatorio(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer Colorectal',
        'RUT': resultado_filtrado['NRO_IDENTIFICADOR_PACIENTE'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['PACIENTE'],
        
        'N° Telefono 1': resultado_filtrado.get('TELEFONO_MOVIL', np.nan),
        'N° Telefono 2':resultado_filtrado.get('TELEFONO_FIJO_1', np.nan),
        'N° Telefono 3': resultado_filtrado.get('TELEFONO_FIJO_2', np.nan),
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
   
    return nuevo_dataframe

def crear_dataframe_ieeh(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer Colorectal',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOMBRES']+ ' ' + resultado_filtrado['APELL_PATE']+ ' '+ resultado_filtrado['APELL_MATE'] ,
        'N° Telefono 1': resultado_filtrado['TELEFONO'],
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe



def crear_dataframe_urgencias(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer Colorectal',
        'RUT': resultado_filtrado['RUT'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['NOMBRES_PACIENTE']+ ' ' + resultado_filtrado['APELLIDOS_PACIENTE'],
        'N° Telefono 1': np.nan,
        'N° Telefono 2': np.nan,
        'N° Telefono 3': np.nan,
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def crear_dataframe_inter(resultado_filtrado):
    nuevo_dataframe = pd.DataFrame({
        'Problema de Salud': 'Cancer Colorectal',
        'RUT': resultado_filtrado['NRO_IDENTIFICADOR'],
        'DV': resultado_filtrado['DV'],
        'Nombre del Paciente':resultado_filtrado['APELLPAT_PAC']+' '+resultado_filtrado['APELLMAT_PAC']+' '+resultado_filtrado['NOM_PAC'],
        'N° Telefono 1': resultado_filtrado['FONO_MOVIL'],
        'N° Telefono 2': resultado_filtrado['FONO_FIJO_1'],
        'N° Telefono 3': resultado_filtrado['FONO_FIJO_2'],
        'Fecha Proximo Hito': np.nan,
        #'Observaciones del Diagnostico': np.nan,
        #'Diagnostico CIE-10': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })

    return nuevo_dataframe

def safe_convert_to_int(value):
    try:
        return int(value)
    except ValueError:
        return value

def merge_ambulatorio(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)


    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.
    
    return merged_df 

def merge_urgencia(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def merge_inter(resultados, nuevo_dataframe):
    df1 = resultados[['RUT','DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO']]
    df2 = nuevo_dataframe

    df1['RUT'] = df1['RUT'].astype(str)
    df2['RUT'] = df2['RUT'].astype(str)

    df1['RUT'] = df1['RUT'].apply(safe_convert_to_int)
    df2['RUT'] = df2['RUT'].apply(safe_convert_to_int)

    # Realizar merge por la columna 'RUT'
    merged_df = pd.merge(df1, df2, on='RUT', how='outer')

    # El parámetro 'on' especifica la columna por la cual se realiza el merge
    # El parámetro 'how' especifica el tipo de merge. 'inner' conserva solo las filas que tienen un RUT que está presente en ambos DataFrames.

    return merged_df

def filtrar_columnas_relevantes(merged_df):
    nuevo_dataframe = merged_df.drop_duplicates(subset='RUT', keep='first')
    nuevo_dataframe = nuevo_dataframe[['Problema de Salud','RUT' ,'DV',
       'Nombre del Paciente', 'N° Telefono 1', 'N° Telefono 2',
       'N° Telefono 3', 'Fecha Proximo Hito', 'DIAGNOSTICOS', 'CODIGOS_DIAGNOSTICO',
       'Nombre agenda Medico especialista']]
    nuevo_dataframe = nuevo_dataframe.rename(columns={'Problema de Salud': 'Problema Oncologico'})
    nuevo_dataframe['Nombre garantia'] = np.nan
    nuevo_dataframe= nuevo_dataframe[['Problema Oncologico',
    'RUT',
    'DV',
    'Nombre del Paciente',
    'N° Telefono 1',
    'N° Telefono 2',
    'N° Telefono 3',
    'Nombre garantia',
    'Fecha Proximo Hito',
    'DIAGNOSTICOS',
    'CODIGOS_DIAGNOSTICO',
    'Nombre agenda Medico especialista']]
    return nuevo_dataframe

def crear_dataframe_ges(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'Problema Oncologico': dataframe_tiroides['Problema de salud'],
        'RUT': dataframe_tiroides['RUT'],
        'DV': dataframe_tiroides['DV'],
        'Nombre del Paciente':dataframe_tiroides['Nombre'],
        'N° Telefono 1': dataframe_tiroides['Telefonos.x'],
        'N° Telefono 2': dataframe_tiroides['Telefonos.y'],
        'N° Telefono 3': np.nan,
        'Nombre garantia': dataframe_tiroides['Nombre garantia'],
        'Fecha Proximo Hito': np.nan,
        'DIAGNOSTICOS': np.nan,
        'CODIGOS_DIAGNOSTICO': np.nan,
        'Fecha Proximo Hito': np.nan,
        'Nombre agenda Medico especialista': np.nan,
    })
    return nuevo_dataframe_2

def crear_dataframe_ultima_atencion(dataframe_tiroides):
    # Crear un nuevo DataFrame con las columnas deseadas
    nuevo_dataframe_2 = pd.DataFrame({
        'RUT': dataframe_tiroides['RUT'],
        'Fecha Ultima atencion': dataframe_tiroides['HORA_TERMINO_ATENCION'],
        'Profesional Ultima atencion': dataframe_tiroides['ESTAMENTO_PROFESIONAL_ATENCION']+ ' '+
        dataframe_tiroides['NOMBRE_PROFESIONAL_ATENCION']+ ' ' + dataframe_tiroides['DESCRIPCION'] + ' '+
        dataframe_tiroides['ESPECIALIDAD_AGENDA']
    })
    return nuevo_dataframe_2

def proceso_dataframe(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides(dataframe)
        resultado_filtrado = agregar_dv(resultado_filtrado)
        resultados = agregar_diagnosticos_y_codigos(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_ambulatorio(resultado_filtrado)
        merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales 

def proceso_dataframe_ultimo(ruta_archivo):
    dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
    resultado_filtrado = filtro_dataframe_ca_tiroides(dataframe)
    resultado_filtrado = agregar_dv(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_ambulatorio(resultado_filtrado)
    merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
    return nuevo_dataframe

def proceso_dataframe_ieeh(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ieeh(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides_ieeh(dataframe)
        resultados = agregar_diagnosticos_ieeh(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_ieeh(resultado_filtrado)
        merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales

def proceso_dataframe_urgencias(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ieeh(ruta_archivo)
        resultado_filtrado = filtro_dataframe_ca_tiroides_urgencias(dataframe)
        resultado_filtrado = agregar_dv_urgencias(resultado_filtrado)
        resultados = agregar_diagnosticos_y_codigos_urgencias(resultado_filtrado)
        nuevo_dataframe = crear_dataframe_urgencias(resultado_filtrado)
        merged_df = merge_urgencia(resultados, nuevo_dataframe)
        nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
        resultados_totales.append(nuevo_dataframe)
    return resultados_totales

def proceso_dataframe_inter(ruta_archivo):
    dataframe = cargar_dataframe_ieeh(ruta_archivo)
    dataframe['FECHA_SOLIC'] = pd.to_datetime(dataframe['FECHA_SOLIC'], format='%d/%m/%Y')
    dataframe = dataframe[dataframe['ESTADO_SOL'].isin(['Solicitada','Pendiente de Autorizar', 'Citada'])]
    dataframe = dataframe[~dataframe['NRO_IDENTIFICADOR'].str.contains('AT')]
    # Definir la fecha de inicio para el filtro
    fecha_hoy = datetime.today()
    fecha_inicio = fecha_hoy - timedelta(days=10)

    fecha_inicio = pd.to_datetime(fecha_inicio, format='%d/%m/%Y')
    # Filtrar el DataFrame
    dataframe = dataframe[dataframe['FECHA_SOLIC'] >= fecha_inicio]
    resultado_filtrado = filtro_dataframe_ca_tiroides_inter(dataframe)
    resultado_filtrado = agregar_dv_inter(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos_inter(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_inter(resultado_filtrado)
    merged_df = merge_urgencia(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
 
    return nuevo_dataframe

def ultimo_csv_en_carpeta(ruta):
    # Obtener lista de archivos en la carpeta
    archivos = os.listdir(ruta)
    
    # Filtrar archivos que sean .csv
    archivos_csv = [archivo for archivo in archivos if archivo.endswith('.csv')]
    
    if not archivos_csv:
        return None  # Si no hay archivos .csv en la carpeta
    
    # Ordenar la lista de archivos por fecha de modificación
    archivos_csv.sort(key=lambda x: os.path.getmtime(os.path.join(ruta, x)), reverse=True)
    
    # Devolver la ruta completa del último archivo .csv
    return os.path.join(ruta, archivos_csv[0])

def convertir_fecha(fecha):
    try:
        return pd.to_datetime(fecha, format='%d/%m/%Y %H:%M')
    except ValueError:
        return pd.to_datetime(fecha, format='%d-%m-%y %H:%M')
    
def obtener_fechas_amb(rutas_archivos):
    resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
    for ruta_archivo in rutas_archivos:
        dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
        dataframe['DV'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[-1]
        dataframe['RUT'] = dataframe['NRO_IDENTIFICADOR_PACIENTE'].str[:-1]
        



        resultados_totales.append(dataframe)
        
    dataframe = pd.concat(resultados_totales)

    
    # Filtrar por las especialidades de Endocrinología Adulto y Endocrinología Infantil
    endocrinologia_df = dataframe[dataframe['ESPECIALIDAD_AGENDA'].isin(['Cirugía Proctológica', 'Imagenología AMB'])]
    endocrinologia_df = endocrinologia_df[endocrinologia_df['ESTAMENTO_PROFESIONAL_ATENCION']=='Médico']

    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
    endocrinologia_df = endocrinologia_df[~endocrinologia_df['ESTADO_CITA'].isin(['No realizada', 'Anulada'])]
    # Convertir la columna 'HORA_TERMINO_ATENCION' a formato de fecha y hora
    #endocrinologia_df['HORA_TERMINO_ATENCION'] = pd.to_datetime(endocrinologia_df['HORA_TERMINO_ATENCION'], format='mixed')
    endocrinologia_df['HORA_TERMINO_ATENCION'] = endocrinologia_df['HORA_TERMINO_ATENCION'].apply(convertir_fecha)

    # Agrupar por RUT y obtener la última fecha de la hora de atención para cada grupo
    #ultima_fecha_por_rut = endocrinologia_df.groupby('RUT').agg({'HORA_TERMINO_ATENCION': 'max'}).reset_index()

    ultima_fecha_por_rut =  endocrinologia_df.groupby('RUT').agg({
    'HORA_TERMINO_ATENCION': 'max',
    'NOMBRE_PROFESIONAL_ATENCION': 'first',
    'ESTAMENTO_PROFESIONAL_ATENCION': 'first',
    'DESCRIPCION': 'first',
    'ESPECIALIDAD_AGENDA': 'first'
}).reset_index()
    # Fusionar el DataFrame original con el DataFrame resultante para recuperar NOMBRE_PROFESIONAL_ATENCION y ESTAMENTO_PROFESIONAL_ATENCION
    resultado = ultima_fecha_por_rut.merge(endocrinologia_df[['RUT', 'HORA_TERMINO_ATENCION', 'NOMBRE_PROFESIONAL_ATENCION', 'ESTAMENTO_PROFESIONAL_ATENCION','DESCRIPCION']], on=['RUT', 'HORA_TERMINO_ATENCION'], how='left')

    return ultima_fecha_por_rut 

# Ruta archivos

In [14]:
dir_actual = os.getcwd()
dir_archivo_csv = os.path.join(dir_actual,'..','Reporteria.lnk')

In [15]:
import win32com.client

def get_target_path(lnk_file):
    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(lnk_file)
    return shortcut.Targetpath

# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Reporteria.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_reporteria = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_reporteria)

In [16]:
# Ruta del archivo de acceso directo (.lnk)
ruta_del_lnk = os.path.join(dir_actual,'..','Unidad_Gestion_Del_Paciente.lnk')
# Obtener la ruta de destino del archivo de acceso directo
ruta_ges = get_target_path(ruta_del_lnk)

#print("Ruta de destino:", ruta_ges)

In [17]:


#################### RUTA ARCHIVOS ##########################################

dias_reportes = 10

# Lista de rutas de archivos
rutas_archivos = []

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)
# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)


#Reportes Ambulatorios Histórico
ruta_archivo_4 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', '202401_Reporte_Ambulatorio_Mensual.csv')
ruta_archivo_5 = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '20240401_Reporte_Ambulatorio_Diario.csv')


# Lista de rutas de archivos
rutas_archivos_hist = [ruta_archivo_4, ruta_archivo_5]
#agregamos los dias 
for i in range(1,int(day_formatted)+1):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_Abril', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', f'{today.year}{month_formatted}{numero_formateado}_Reporte_Ambulatorio_Diario.csv'))
for i in range(1,today.month):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Mensual Ambulatorio', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Mensual', f'{today.year}{numero_formateado}_Reporte_Ambulatorio_Mensual.csv'))

for i in range(1,6):
        # Esto asegura que el número tenga dos dígitos, agregando un cero inicial si es necesario
    numero_formateado = f'{i:02}'
    #rutas_archivos.append(os.path.join(dir_datos_hlf, 'BDD_A_M', 'Anual Ambulatorio', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
    rutas_archivos_hist.append(os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Anual', f'{today.year-i}_Reporte_Ambulatorio_Anual.csv'))
  



#Reportes Ambulatorios


patron_amb = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','Diario', '*Reporte_Ambulatorio_Diario*')

archivos_amb = glob.glob(patron_amb)

archivos_ordenados_amb = sorted(archivos_amb, key=os.path.getmtime, reverse = True)

rutas_archivos = archivos_ordenados_amb[:dias_reportes]



#Reportes IEEH

rutas_archivos_2 = []


patron_ieeh = os.path.join(ruta_reporteria,'Reporte_IEEH','Diario', '*Informe_Diario-IEEE*')

archivos_ieeh  = glob.glob(patron_ieeh )

archivos_ordenados_ieeh  = sorted(archivos_ieeh , key=os.path.getmtime, reverse = True)

rutas_archivos_2 = archivos_ordenados_ieeh[:dias_reportes]


#Reportes Urgencias


rutas_archivos_3 = []


patron_urg = os.path.join(ruta_reporteria,'Reportes_Produccion_Urgencia','Diario', '*Urgencias_atenciones*')

archivos_urg  = glob.glob(patron_urg)

archivos_ordenados_urg  = sorted(archivos_urg , key=os.path.getmtime, reverse = True)

rutas_archivos_3 = archivos_ordenados_urg[:dias_reportes]





In [18]:
#Reportes SIC

patron_sic = os.path.join(ruta_reporteria,'Reportes_Solicitudes','Mensual', '*Informe_Solicitudes_DerivacionCE*')

archivos_sic  = glob.glob(patron_sic)

archivos_ordenados_sic  = sorted(archivos_sic , key=os.path.getmtime, reverse = True)

rutas_archivos_4 = archivos_ordenados_sic[:1]


In [19]:
rutas_archivos_4

['\\\\apolo\\Reporteria\\Reportes_Solicitudes\\Mensual\\20240708_Informe_Solicitudes_DerivacionCE.csv']

# Ver casos raros

In [20]:
resultados_totales = []  # Lista para almacenar los resultados de todos los archivos
for ruta_archivo in rutas_archivos:
    dataframe = cargar_dataframe_ambulatorio(ruta_archivo)
    resultado_filtrado = filtro_dataframe_ca_tiroides(dataframe)
    resultado_filtrado = agregar_dv(resultado_filtrado)
    resultados = agregar_diagnosticos_y_codigos(resultado_filtrado)
    nuevo_dataframe = crear_dataframe_ambulatorio(resultado_filtrado)
    merged_df = merge_ambulatorio(resultados, nuevo_dataframe)
    nuevo_dataframe = filtrar_columnas_relevantes(merged_df)
    resultados_totales.append(nuevo_dataframe)


AttributeError: Can only use .str accessor with string values!

In [ ]:
aux = pd.concat(resultados_totales)

In [ ]:
aux

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
0,Cancer Colorectal,6611957,2,GLORIA BEATRIZ BECERRA BECERRA,57703638,2898750,NaN,NaN,NaN,"[Insuficiencia mitral reumatica, Hipertension ...","[I50, C20, I051, E039, C50, I10]",NaN
0,Cancer Colorectal,10199797,9,ALEJANDRA MATILDE SEPÚLVEDA YÁÑEZ,935648720,979606444,920813494.0,NaN,NaN,[Tumor maligno del colon],[C18],NaN
1,Cancer Colorectal,11304779,8,BRAULIO ESTEBAN CALFUÑANCO PICHINAO,963240021,NaN,NaN,NaN,NaN,"[METASTASIS HEPATICAS, CANCER DE COLON]",[],NaN
2,Cancer Colorectal,15701292,4,JOSSELLIN TIARE TORRES MACHUCA,982457877,933366060,NaN,NaN,NaN,[Tumor maligno del colon],[C18],NaN
3,Cancer Colorectal,4703370,5,MARÍA TERESA LAZO FUENZALIDA,56986532896,56934372248,942679095.0,NaN,NaN,"[Fractura pertrocanteriana, Tumor maligno del ...","[D50, M16, Z933, C18, S721]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1,Cancer Colorectal,13168174,7,FRANCIA VALESKA MELLA RUIZ,NaN,954973714,931420207.0,NaN,NaN,[Tumor maligno del recto],[C20],NaN
2,Cancer Colorectal,13395033,8,JACOB PASCUAL LINCOPÁN MARILEO,955387181,956725363,945083431.0,NaN,NaN,"[GIST GASTRICO, Tumor maligno del estomago, TU...",[C16],NaN
3,Cancer Colorectal,6394069,0,JORGE ALFREDO MOTTO DEVIA,0963172315,020995301592,NaN,NaN,NaN,[CANCER DE RECTO],[],NaN
4,Cancer Colorectal,6682145,5,ELIANA DEL CARMEN BRIZUELA SÁNCHEZ,56989164896,56956398381,56931081413.0,NaN,NaN,"[Tumor maligno de los bronquios y del pulmon, ...",[C34],NaN


# Generar dataframes

In [ ]:
# Llamar a la función proceso_dataframe con la lista de rutas de archivos
resultado = obtener_fechas_amb(rutas_archivos_hist)

resultado_final = proceso_dataframe(rutas_archivos)

resultado_final_2 = proceso_dataframe_ieeh(rutas_archivos_2)

resultado_final_3 = proceso_dataframe_urgencias(rutas_archivos_3)

resultado_final_4 = proceso_dataframe_inter(rutas_archivos_4[0])

In [ ]:

#os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')
ruta_del_archivo = os.path.join(ruta_ges,'UNIDAD GES', 'SCRIPT_GES','Cruce GES','Cruce.xlsx')


# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

In [ ]:
# Carga el archivo XLSX en un DataFrame
dataframe = pd.read_excel(ruta_del_archivo)

# Construir la ruta completa al archivo CSV
dataframe['RUT']= dataframe['RUT'].apply(lambda x : str(x))
dataframe['RUT COMPLETO'] = dataframe['RUT'].astype(str)  + dataframe['DV'].astype(str)
dataframe_tiroides = dataframe[dataframe['Problema de salud'] == 'Cáncer Colorectal . {decreto n° 4/2013}']

dataframe_tiroides["Fecha inicio"]=pd.to_datetime(dataframe_tiroides["Fecha inicio"])
# Ordenar en orden descendente por la columna "FECHA_CITA"
dataframe_tiroides = dataframe_tiroides.sort_values(by='Fecha inicio', ascending=False)


# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"
dataframe_tiroides = dataframe_tiroides.drop_duplicates(subset='RUT', keep='first')

nuevo_dataframe_2 = crear_dataframe_ges(dataframe_tiroides)



resultado_final.append(nuevo_dataframe_2)



#if ultimo_csv:
#    print("La dirección del último archivo .csv en la carpeta es:", ultimo_csv)
    
#    resultado_final.append(proceso_dataframe_ultimo(ultimo_csv))
#    df_concatenated = pd.concat(resultado_final)

#else:
#    print("No se encontraron archivos .csv en la carpeta.")
#    df_concatenated = pd.concat(resultado_final)
 # Concatenar los DataFrames
resultado_final.extend(resultado_final_2)
resultado_final.extend(resultado_final_3)
resultado_final.extend([resultado_final_4])

df_concatenated = pd.concat(resultado_final)






# Eliminar duplicados basados en la columna "NRO_IDENTIFICADOR_PACIENTE"

#consolidated_df = df_concatenated.groupby('RUT').apply(lambda x: x.fillna(method='ffill').fillna(method='bfill')).drop_duplicates().reset_index(drop=True)

#resultado_filtrado_2 = consolidated_df.drop_duplicates(subset='RUT', keep='first')

resultado_filtrado_2 = df_concatenated.drop_duplicates(subset='RUT', keep='first')

# Iterar sobre las filas del primer DataFrame
for index, row in resultado_filtrado_2.iterrows():
    rut = row['RUT']
    
    # Buscar el valor correspondiente en el segundo DataFrame
    nombre_garantia = nuevo_dataframe_2.loc[nuevo_dataframe_2['RUT'] == str(rut), 'Nombre garantia'].values
    
    # Encontrar la fila correspondiente en el primer DataFrame y asignar el valor
    resultado_filtrado_2.loc[resultado_filtrado_2['RUT'] == rut, 'Nombre garantia'] = nombre_garantia[0] if nombre_garantia.size > 0 else None

resultado_filtrado_2['RUT'] = resultado_filtrado_2['RUT'].apply(lambda x: str(x))

In [ ]:
resultado_filtrado_2 = resultado_filtrado_2.drop_duplicates(subset='RUT', keep='first')

In [ ]:
df_concatenated

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
0,Cancer Colorectal,12566341,9,JUAN CARLOS FUENTES ULLOA,946223820,965221313,NaN,NaN,NaN,"[Embolia pulmonar, Tumor maligno del recto, Ot...","[C20, I82, I26, N10, N17, L89]",NaN
1,Cancer Colorectal,12893228,3,JEAN MICHEL BRAVO POBLETE,0940326409,0978477345,936841950.0,NaN,NaN,"[Polipo del colon, COLECTOMIA TOTAL, Tumor de ...","[K635, D374]",NaN
2,Cancer Colorectal,4703370,5,MARÍA TERESA LAZO FUENZALIDA,56986532896,56934372248,942679095.0,NaN,NaN,"[Fractura pertrocanteriana, Tumor maligno del ...","[C18, S721, M16, Z933, D50]",NaN
4,Cancer Colorectal,6434777,2,GEORGINA ESMERALDA GUERRA FUENZALIDA,0979661179,973325376,NaN,NaN,NaN,"[Tumor maligno de la vesicula biliar, Tumor ma...","[C20, C23]",NaN
0,Cancer Colorectal,10128916,8,HÉCTOR GARCÍA VILLA,934339513,922515061,NaN,NaN,NaN,[Tumor maligno del colon ascendente],[C182],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
37,Cancer Colorectal,7872932,5,LÓPEZ PIZARRO DAVID MIGUEL,972034848,000000000,972946156,NaN,NaN,"[NODULO PULMONAR, SOSPECHA TUMOR DE COLON ]",[],NaN
38,Cancer Colorectal,8089466,K,PARIS RODRÍGUEZ PATRICIO,977394489,991740981,978362791,NaN,NaN,[Tumor maligno del recto],[C20],NaN
39,Cancer Colorectal,8110833,1,LASTRA HERNÁNDEZ SONIA DEL ROSARIO,89737962,95451332,NaN,NaN,NaN,[cancer de colon sigmoides ],[],NaN
40,Cancer Colorectal,9804764,6,PINO TOLEDO VÍCTOR ALBERTO,079021837,79021856,NaN,NaN,NaN,[CANCER DE COLON],[],NaN


In [ ]:
subcadena = '7186507'

In [ ]:
df_concatenated[df_concatenated['RUT'].astype(str).str.contains(subcadena)]

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,CODIGOS_DIAGNOSTICO,Nombre agenda Medico especialista
15,Cancer Colorectal,7186507,K,MARÍA MOZÓ CHEUQUENAO,90000264,29846572,NaN,NaN,NaN,"[Tumor maligno del colon, Hemorragia del ano y...","[K625, C18]",NaN


In [ ]:
#nombre_archivo = 'Total pacientes_nuevos_ca_colorectal.xlsx'  # Nombre del archivo de Excel
#resultado_filtrado_2.to_excel(nombre_archivo, index=False)  # index=False para no incluir el índice en el archivo de Excel


# Cruce BDD Gestoras

In [ ]:
dir_actual = os.getcwd()
os.listdir(os.path.join(dir_actual, '..','CA COLORECTAL'))

['BASE TRABAJO GESTION ONCOLOGICA_CANCER COLORECTAL.xlsx',
 'Reporte diario casos nuevos',
 '~$BASE TRABAJO GESTION ONCOLOGICA_CANCER COLORECTAL.xlsx']

In [ ]:
archivo_excel = os.path.join(dir_actual, '..','CA COLORECTAL', 'BASE TRABAJO GESTION ONCOLOGICA_CANCER COLORECTAL.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


In [ ]:
dataframe_2

,Problema Oncologico,Rut,DV,Nombre,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cáncer Colorectal,15701292,4,"TORRES MACHUCA, JOSSELLIN TIARE",Seguimiento,Pendiente Examen,2024-06-04 00:00:00,Dr Rojas,NaN,16/05/2024 en ultimo control 07/05/2024 indica...,NaN,NaN,04-06-2024,DR. JULIO IGNACIO ROJAS ASTORGA,03-12-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
1,Cáncer Colorectal,7629228,0,"AEDO MUÑOZ, MARÍA JEANNETTE",Ingreso,Agendado Médico,04-06-2024,Dr Tica,NaN,NaN,NaN,- / - / / / 30759801 / - / -/.,04-06-2024,Dr. ANIBAL TICA PLAYA,NaN,NaN
2,Cáncer Colorectal,7029609,8,"ALIAGA NAVARRO, CARLOS VALENTÍN",Seguimiento,Agendado Médico,04-06-2024,Dr Garcia,NaN,16/05/2024 en ultimo control 09/01/2024 indica...,NaN,NaN,04-06-2024,DR. JOSÉ ANTONIO HERNÁNDEZ MONTALVO,01-09-2024,Médico DIEGO RICARDO GARCÍA PRADO Control - Co...
3,Cáncer Colorectal,8089466,K,PATRICIO PARIS RODRÍGUEZ,NaN,NaN,04-06-2024,Dra Carrillo,NaN,"16/05/2024 CASO ESPCIAL, DADO DE ALTA POR Dra ...",NaN,977394489/991740981/978362791,04-06-2024,DR. JULIO IGNACIO ROJAS ASTORGA,05-07-2024,Médico KATYA MARINA CARRILLO GONZÁLEZ TC de Ab...
4,Cáncer Colorectal,7844300,6,"HIDALGO MADRIAGA, JEREMÍAS DEL ROSARIO",Seguimiento,Pendiente Examen,05-06-2024,Dr Marin,NaN,16/05/2024 EN ULTIMO CONTROL DR SOLCIITA COLON...,NaN,NaN,05-06-2024,BECADO DR. DIEGO IGNACIO MARÍN GARAY,13-03-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,Cáncer Colorectal . {decreto n° 4/2013},6787461,7,"JIMÉNEZ PAVEZ, MARÍA DEL CARMEN",NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 93265414 / / 90696465 / 97364131 / -/...,NaN,NaN,03-10-2024,Médico CRISTINA ANDREA RIVERA ARCOS Ecografia ...
1889,Cancer Colorectal,7107748,9,JUAN MANUEL GONZÁLEZ CORRALES,NaN,NaN,NaN,NaN,NaN,NaN,"['Otras poliartrosis', 'Hiperplasia de la pros...",56950166803/56987939738/-,NaN,NaN,19-04-2024,Médico MANUEL ALEJANDRO PEÑALOZA HERRERA TC de...
1890,Cáncer Colorectal . {decreto n° 4/2013},7613229,1,"PINO SAAVEDRA, MOISÉS ARSENIO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 24077326 / / 66644758 / - / -/9966447...,NaN,NaN,NaN,NaN
1891,Cancer Colorectal,9864638,8,MIGUEL ANGEL CERNA OSORIO,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno del colon'],0978751413/027553567/982523326,NaN,NaN,05-07-2024,Médico Zeyuan Zhang AngioTAC de Abdomen Image...


In [ ]:

dataframe = resultado_filtrado_2.copy()

# Obtener la ruta del directorio actual del script
#dir_actual = os.path.dirname(__file__)
dir_actual = os.getcwd()


archivo_excel = os.path.join(dir_actual, '..','CA COLORECTAL', 'BASE TRABAJO GESTION ONCOLOGICA_CANCER COLORECTAL.xlsx') # Gestion_Oncologica -> apolo

dataframe_2 = pd.read_excel(archivo_excel, sheet_name='Data')


# Obtener la cantidad de filas que no son nulas en la columna 'mi_columna'
cantidad_filas_no_nulas = dataframe_2['Rut'].count()



print("Cantidad de pacientes en la base", cantidad_filas_no_nulas)



dataframe_2 = dataframe_2[:cantidad_filas_no_nulas]

dataframe_2.reset_index(drop=True, inplace=True)


# Renombrar la columna 'old_name' a 'new_name'
dataframe_2 = dataframe_2.rename(columns={'Rut': 'RUT'})


dataframe_2 = dataframe_2.rename(columns={'Problema de Salud': 'Problema Oncologico'})

dataframe_2 = dataframe_2.rename(columns={'Nombre': 'Nombre del Paciente'})

# Definir una función para quitar los últimos dos dígitos
def quitar_ultimos_dos(elemento):
    return np.int64(elemento[:-2])

# Convertir todos los elementos de la columna 'RUT' a string
dataframe_2['RUT'] = dataframe_2['RUT'].astype(str)

# Aplicar la función a la columna 'RUT'
#dataframe_2['RUT'] = dataframe_2['RUT'].apply(quitar_ultimos_dos)



# Identificar las filas donde 'DV' es NaN
filas_dv_nan = dataframe_2[dataframe_2['DV'].isna()]

# Iterar sobre estas filas y corregir los valores de 'DV' y 'Rut'
for index, fila in filas_dv_nan.iterrows():
    rut = str(fila['RUT'])
    if True:
      dv = rut[-1]  # Último dígito del Rut
      dataframe_2.at[index, 'DV'] = dv
      dataframe_2.at[index, 'RUT'] = rut[:-1]  # Eliminar el último dígito del Rut
dataframe_2['RUT'] = dataframe_2['RUT'].astype(float)
dataframe_2['RUT'] = dataframe_2['RUT'].astype(int)


Cantidad de pacientes en la base 1893


In [ ]:
dataframe_2

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
0,Cáncer Colorectal,15701292,4,"TORRES MACHUCA, JOSSELLIN TIARE",Seguimiento,Pendiente Examen,2024-06-04 00:00:00,Dr Rojas,NaN,16/05/2024 en ultimo control 07/05/2024 indica...,NaN,NaN,04-06-2024,DR. JULIO IGNACIO ROJAS ASTORGA,03-12-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
1,Cáncer Colorectal,7629228,0,"AEDO MUÑOZ, MARÍA JEANNETTE",Ingreso,Agendado Médico,04-06-2024,Dr Tica,NaN,NaN,NaN,- / - / / / 30759801 / - / -/.,04-06-2024,Dr. ANIBAL TICA PLAYA,NaN,NaN
2,Cáncer Colorectal,7029609,8,"ALIAGA NAVARRO, CARLOS VALENTÍN",Seguimiento,Agendado Médico,04-06-2024,Dr Garcia,NaN,16/05/2024 en ultimo control 09/01/2024 indica...,NaN,NaN,04-06-2024,DR. JOSÉ ANTONIO HERNÁNDEZ MONTALVO,01-09-2024,Médico DIEGO RICARDO GARCÍA PRADO Control - Co...
3,Cáncer Colorectal,8089466,K,PATRICIO PARIS RODRÍGUEZ,NaN,NaN,04-06-2024,Dra Carrillo,NaN,"16/05/2024 CASO ESPCIAL, DADO DE ALTA POR Dra ...",NaN,977394489/991740981/978362791,04-06-2024,DR. JULIO IGNACIO ROJAS ASTORGA,05-07-2024,Médico KATYA MARINA CARRILLO GONZÁLEZ TC de Ab...
4,Cáncer Colorectal,7844300,6,"HIDALGO MADRIAGA, JEREMÍAS DEL ROSARIO",Seguimiento,Pendiente Examen,05-06-2024,Dr Marin,NaN,16/05/2024 EN ULTIMO CONTROL DR SOLCIITA COLON...,NaN,NaN,05-06-2024,BECADO DR. DIEGO IGNACIO MARÍN GARAY,13-03-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,Cáncer Colorectal . {decreto n° 4/2013},6787461,7,"JIMÉNEZ PAVEZ, MARÍA DEL CARMEN",NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 93265414 / / 90696465 / 97364131 / -/...,NaN,NaN,03-10-2024,Médico CRISTINA ANDREA RIVERA ARCOS Ecografia ...
1889,Cancer Colorectal,7107748,9,JUAN MANUEL GONZÁLEZ CORRALES,NaN,NaN,NaN,NaN,NaN,NaN,"['Otras poliartrosis', 'Hiperplasia de la pros...",56950166803/56987939738/-,NaN,NaN,19-04-2024,Médico MANUEL ALEJANDRO PEÑALOZA HERRERA TC de...
1890,Cáncer Colorectal . {decreto n° 4/2013},7613229,1,"PINO SAAVEDRA, MOISÉS ARSENIO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 24077326 / / 66644758 / - / -/9966447...,NaN,NaN,NaN,NaN
1891,Cancer Colorectal,9864638,8,MIGUEL ANGEL CERNA OSORIO,NaN,NaN,NaN,NaN,NaN,NaN,['Tumor maligno del colon'],0978751413/027553567/982523326,NaN,NaN,05-07-2024,Médico Zeyuan Zhang AngioTAC de Abdomen Image...


# Merge Total pacientes vs Gestoras

In [ ]:
# Ejemplo de creación de DataFrame (puedes cargar tus propios DataFrames)
df1 = dataframe.copy()
df2 = dataframe_2.copy()

# Agrega una columna para identificar el origen de cada RUT
df1['_df'] = 'df1'
df2['_df'] = 'df2'

# Convertir la columna 'RUT' en ambos DataFrames al mismo tipo de datos
df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)

#print(f"df_1: {df1['RUT'] }")
#print(f"df_2: {df2['RUT'] }")

# Realiza un merge de los dos DataFrames en la columna 'RUT' y conserva solo las filas que aparecen en 'left_only'
df_merge = pd.merge(df1, df2, on='RUT', how='outer', indicator=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Nombre del Paciente_x'].fillna(df_merge['Nombre del Paciente_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Nombre del Paciente_x': 'Nombre del Paciente'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Nombre del Paciente_y'], inplace=True)

# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['Problema Oncologico_x'].fillna(df_merge['Problema Oncologico_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'Problema Oncologico_x': 'Problema Oncologico'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['Problema Oncologico_y'], inplace=True)


# Combinar las columnas 'DV_x' y 'DV_y' en una sola columna 'DV'
df_merge['DV_x'].fillna(df_merge['DV_y'], inplace=True)

# Renombrar la columna 'DV_x' a 'DV'
df_merge.rename(columns={'DV_x': 'DV'}, inplace=True)

# Eliminar la columna 'DV_y' si ya no es necesaria
df_merge.drop(columns=['DV_y'], inplace=True)

df_ca_tiroide = df_merge[df_merge['_merge']=='left_only']

# Reemplazar los valores NaN por '-'
df_ca_tiroide['N° Telefono 1'] = df_ca_tiroide['N° Telefono 1'].fillna('-')
df_ca_tiroide['N° Telefono 2'] = df_ca_tiroide['N° Telefono 2'].fillna('-')
df_ca_tiroide['N° Telefono 3'] = df_ca_tiroide['N° Telefono 3'].fillna('-')

def combinar_telefonos(row):
    return '/'.join(filter(None, [str(row['N° Telefono 1']), str(row['N° Telefono 2']), str(row['N° Telefono 3'])]))

# Aplicar la función a lo largo de las filas del DataFrame para crear la columna 'Telefono Consolidado'
df_ca_tiroide_telefono = df_ca_tiroide[['N° Telefono 1','N° Telefono 2','N° Telefono 3']]
df_ca_tiroide['Telefono Consolidado2'] = df_ca_tiroide_telefono.apply(combinar_telefonos, axis=1)

# Eliminar las columnas originales de los números de teléfono si ya no son necesarias
df_ca_tiroide.drop(columns=['N° Telefono 1', 'N° Telefono 2', 'N° Telefono 3'], inplace=True)

df_ca_tiroide['Etapa'] = np.nan

df_ca_tiroide['Estado'] = np.nan

df_ca_tiroide['Fecha Proximo Hito'] = np.nan

df_ca_tiroide['Observacion'] = np.nan

df_ca_tiroide['Medico Responsable'] = np.nan

df_ca_tiroide['Fecha Ultima atencion'] = np.nan

df_ca_tiroide['Profesional Ultima atencion'] = np.nan

df_ca_tiroide['MEDICO'] = np.nan

df_ca_tiroide['CITAS FUTURAS'] = np.nan

#df_ca_tiroide = df_ca_tiroide[['Problema Oncologico', 'RUT', 'DV', 'Nombre del Paciente','Etapa','Nombre garantia','Estado','Fecha Proximo Hito','Observacion','Medico Responsable','Fecha Ultima atencion', 'Profesional Ultima atencion','DIAGNOSTICOS', 'Telefono Consolidado2']]
df_ca_tiroide = df_ca_tiroide[['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Etapa',
 'Estado',
 'Fecha Próximo Hito',
 'Observacion',
 'Medico Responsable',
 'DIAGNOSTICOS',
 'Telefono Consolidado2',
 'CITAS FUTURAS',
 'MEDICO',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']]

In [ ]:
df_merge

,Problema Oncologico,RUT,DV,Nombre del Paciente,N° Telefono 1,N° Telefono 2,N° Telefono 3,Nombre garantia,Fecha Proximo Hito,DIAGNOSTICOS,...,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion,_df_y,_merge
0,Cáncer Colorectal,10001217,0,"BELMAR GAVILÁN, WALDO ELEODORO DEL C",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,02-07-2023,Médico KATYA MARINA CARRILLO GONZÁLEZ Control ...,df2,right_only
1,Cáncer Colorectal,10014745,9,ELIANA ANGÉLICA VELÁSQUEZ SEVERINO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-/-/-,NaN,NaN,NaN,NaN,df2,right_only
2,Cáncer Colorectal,10038253,9,HÉCTOR HERNÁN TAPIA LIZANA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,09996642458/022296642468/-,NaN,NaN,02-12-2024,Médico KATYA MARINA CARRILLO GONZÁLEZ Control ...,df2,right_only
3,Cáncer Colorectal,10062717,5,"TORO CARO, AURORA DEL TRÁNSITO",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,28-03-2024,Médico SEBASTIÁN ANDRÉS LÓPEZ NÚÑEZ TC de Abdo...,df2,right_only
4,Cáncer Colorectal . {decreto n° 4/2013},10069096,9,"ROMERO VALENZUELA, MARGARITA ISABEL DEL CARMEN",- / - / 44775429 / / 44775429 / 44775429 / -/.,#ND,NaN,Confirmación,NaN,NaN,...,NaN,NaN,"['Tumor maligno del colon', 'Hemorroides']",66731555/66731555/-,NaN,NaN,30-04-2024,Médico DIEGO RICARDO GARCÍA PRADO Derivacion d...,df2,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897,Cáncer Colorectal,9977981,0,"CASTILLO FUENTES, ADRIANA DEL CARMEN",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,03-04-2024,Médico Ramón Puga Bermudez Control - Consulta ...,df2,right_only
1898,Cáncer Colorectal,9980154,9,EDUVINA AURORA GONZÁLEZ CALDERÓN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,56920081705/56956551090/56945919994,NaN,NaN,22-11-2022,Médico FLORENCIA PINTO GILARDONI TC de Abdomen...,df2,right_only
1899,Cáncer Colorectal,9991146,8,MARISOL DEL CARMEN JUÁREZ PINTO,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,941086803/983915307/-,NaN,NaN,04-11-2024,Médico PABLO ESTEBAN BAEZA IBAÑEZ Control - Co...,df2,right_only
1900,Cáncer Colorectal . {decreto n° 4/2013},9991552,8,"SILVA POVEDA, BELARMINO ENRIQUE",#ND,#ND,NaN,Confirmación,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [ ]:
df_ca_tiroide

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Observacion,Medico Responsable,DIAGNOSTICOS,Telefono Consolidado2,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion
277,Cancer Colorectal,14665475,4,MANUEL GERONIMO MARIN ACOSTA,NaN,NaN,NaN,NaN,NaN,"[Polipo del colon, Tumor maligno del colon, He...",99070779/993454171/-,NaN,NaN,NaN,NaN
589,Cancer Colorectal,4056432,2,GABRIELA DEL CARMEN DURÁN LOYOLA,NaN,NaN,NaN,NaN,NaN,[CA COLON],56040808/22893945/-,NaN,NaN,NaN,NaN
754,Cancer Colorectal,4703370,5,MARÍA TERESA LAZO FUENZALIDA,NaN,NaN,NaN,NaN,NaN,"[Fractura pertrocanteriana, Tumor maligno del ...",56986532896/56934372248/942679095.0,NaN,NaN,NaN,NaN
1036,Cancer Colorectal,5609496,2,MARÍA ELENA ESPINOZA ARCE,NaN,NaN,NaN,NaN,NaN,"[Fibrilacion y aleteo auricular, Otros trastor...",9 8743 9506/9 9872 4765/-,NaN,NaN,NaN,NaN
1267,Cáncer Colorectal . {decreto n° 4/2013},6287440,6,"SOTO CHÁVEZ, RICARDO BONIFACIO",NaN,NaN,NaN,NaN,NaN,NaN,#ND/#ND/-,NaN,NaN,NaN,NaN
1296,Cáncer Colorectal . {decreto n° 4/2013},6384691,0,"DE MONTI PIZARRO, ANGEL PATRICIO",NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22989355 / / 44042321 / - / -/./FIJO:...,NaN,NaN,NaN,NaN
1488,Cancer Colorectal,7186507,K,MARÍA MOZÓ CHEUQUENAO,NaN,NaN,NaN,NaN,NaN,"[Tumor maligno del colon, Hemorragia del ano y...",90000264/29846572/-,NaN,NaN,NaN,NaN
1639,Cáncer Colorectal . {decreto n° 4/2013},8006466,7,"BRICEÑO BRAVO, MANUEL AMÉRICO",NaN,NaN,NaN,NaN,NaN,NaN,- / - / 7758528 / / 52284527 / 27758528 / -/5...,NaN,NaN,NaN,NaN
1900,Cáncer Colorectal . {decreto n° 4/2013},9991552,8,"SILVA POVEDA, BELARMINO ENRIQUE",NaN,NaN,NaN,NaN,NaN,NaN,#ND/#ND/-,NaN,NaN,NaN,NaN


In [ ]:
print(f'Cantidad de pacientes nuevos encontrados: {df_ca_tiroide.shape[0]}')

Cantidad de pacientes nuevos encontrados: 9


In [ ]:
list(df_ca_tiroide.columns)

['Problema Oncologico',
 'RUT',
 'DV',
 'Nombre del Paciente',
 'Etapa',
 'Estado',
 'Fecha Próximo Hito',
 'Observacion',
 'Medico Responsable',
 'DIAGNOSTICOS',
 'Telefono Consolidado2',
 'CITAS FUTURAS',
 'MEDICO',
 'Fecha Ultima atencion',
 'Profesional Ultima atencion']

In [ ]:

df_ca_tiroide['Fecha de Comité'] = np.nan


In [ ]:
lista = ['Problema Oncologico', 'RUT','DV','Nombre del Paciente','Etapa','Estado','Fecha Próximo Hito','Medico Responsable','Fecha de Comité','Observacion','DIAGNOSTICOS','Telefono Consolidado2','CITAS FUTURAS','MEDICO','Fecha Ultima atencion','Profesional Ultima atencion']

# Guardar estadística de llegada

In [ ]:
fecha_hoy = date.today()
data = {'Problema Oncologico': ['Cancer Colorectal'], 'Fecha Medición': [fecha_hoy], 'Cantidad de llegada': [df_ca_tiroide.shape[0]], 'Cantidad de pacientes base': [cantidad_filas_no_nulas]}
estadisticas_llegadas = pd.DataFrame(data)

nombre_archivo = f'Estadisticas_llegadas.xlsx'
archivo_excel = os.path.join(dir_actual, '..','Power BI',f'{nombre_archivo}')

df_existente = pd.read_excel(archivo_excel)
df_existente['Fecha Medición'] = pd.to_datetime(df_existente['Fecha Medición'])


wb = Workbook()
ws = wb.active
df_final = pd.concat([df_existente,estadisticas_llegadas], ignore_index=True)


for r in dataframe_to_rows(df_final, index= False, header = True):
    ws.append(r)

for cell in ws['B'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'Estadisticas_llegadas.xlsx'
wb.save(os.path.join(dir_actual, '..','Power BI',f'{nombre_archivo}'))


# Agregar Fallecidos

In [ ]:
dir_actual = os.getcwd()
dir_gestoras = os.path.join(dir_actual,'..')

In [ ]:
ruta_del_archivo_2 = os.path.join(dir_gestoras, 'FALLECIDOS HLF CORTE 30_04_2024.xlsx')


In [ ]:
# Carga el archivo XLSX en un DataFrame
fallecidos = pd.read_excel(ruta_del_archivo_2)

In [ ]:
fallecidos_rut = pd.DataFrame({'RUT': fallecidos['RUD']})

In [ ]:
import pandas as pd

# Suponiendo que tus dataframes se llaman df_consolidado y df_fallecidos


# Combinamos los dataframes en función de la columna 'RUT'


fallecidos_rut['RUT'] = fallecidos_rut['RUT'].astype(str)  # Convertir la columna a tipo string
fallecidos_rut['RUT'] = fallecidos_rut['RUT'].replace(['-', '\.'], '', regex=True) 

fallecidos_rut['RUT'] = fallecidos_rut['RUT'].astype(str)


In [ ]:
df_pacientes_nuevos = df_ca_tiroide[lista].copy()

In [ ]:
df_pacientes_nuevos

In [ ]:
df_pacientes_nuevos.loc[df_pacientes_nuevos['RUT'].isin(fallecidos_rut['RUT']), 'Estado'] = 'Fallecido'


In [ ]:
df_pacientes_nuevos.loc[df_pacientes_nuevos['RUT'].isin(fallecidos_rut['RUT']), 'Etapa'] = 'Alta'


In [ ]:
df_pacientes_nuevos

In [ ]:
df_pacientes_nuevos = df_pacientes_nuevos[~(df_pacientes_nuevos['RUT']=='nan')]

# Guardar dataframe

In [ ]:
# Guardar el DataFrame en un archivo de Excel

# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)



nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_Pacientes_nuevos_ca_colorectal.xlsx'  # Nombre del archivo de Excel
df_pacientes_nuevos.to_excel(os.path.join(dir_actual, '..','CA COLORECTAL','Reporte diario casos nuevos',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 

# Convertir la columna 'RUT' en DataFrame_2 a tipo object si es necesario


df_concatenated_2 = pd.concat([dataframe_2,df_pacientes_nuevos])

#print(list[df_concatenated_2.columns])




df_concatenated_2['RUT'] = df_concatenated_2['RUT'].astype(str)
resultado['RUT'] = resultado['RUT'].astype(str)
# Fusionar DataFrame_2 con el DataFrame de resultado utilizando la columna 'RUT' como clave de fusión

dataframe_final = df_concatenated_2.merge(resultado, on='RUT', how='left')

#print(list[dataframe_final.columns])


dataframe_final_2 = crear_dataframe_ultima_atencion(dataframe_final)

In [ ]:
df_concatenated_2


,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,Fecha Ultima atencion,Profesional Ultima atencion,DIAGNOSTICOS,Telefono Consolidado2
0,Cáncer Colorectal,15701292,4,"TORRES MACHUCA, JOSSELLIN TIARE",Seguimiento,Pendiente Examen,2024-06-04 00:00:00,Dr Rojas,NaN,16/05/2024 en ultimo control 07/05/2024 indica...,NaN,NaN,04-06-2024,DR. JULIO IGNACIO ROJAS ASTORGA,03-12-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...,NaN,NaN
1,Cáncer Colorectal,7629228,0,"AEDO MUÑOZ, MARÍA JEANNETTE",Ingreso,Agendado Médico,04-06-2024,Dr Tica,NaN,NaN,NaN,- / - / / / 30759801 / - / -/.,04-06-2024,Dr. ANIBAL TICA PLAYA,NaN,NaN,NaN,NaN
2,Cáncer Colorectal,7029609,8,"ALIAGA NAVARRO, CARLOS VALENTÍN",Seguimiento,Agendado Médico,04-06-2024,Dr Garcia,NaN,16/05/2024 en ultimo control 09/01/2024 indica...,NaN,NaN,04-06-2024,DR. JOSÉ ANTONIO HERNÁNDEZ MONTALVO,01-09-2024,Médico DIEGO RICARDO GARCÍA PRADO Control - Co...,NaN,NaN
3,Cáncer Colorectal,8089466,K,PATRICIO PARIS RODRÍGUEZ,NaN,NaN,04-06-2024,Dra Carrillo,NaN,"16/05/2024 CASO ESPCIAL, DADO DE ALTA POR Dra ...",NaN,977394489/991740981/978362791,04-06-2024,DR. JULIO IGNACIO ROJAS ASTORGA,05-07-2024,Médico KATYA MARINA CARRILLO GONZÁLEZ TC de Ab...,NaN,NaN
4,Cáncer Colorectal,7844300,6,"HIDALGO MADRIAGA, JEREMÍAS DEL ROSARIO",Seguimiento,Pendiente Examen,05-06-2024,Dr Marin,NaN,16/05/2024 EN ULTIMO CONTROL DR SOLCIITA COLON...,NaN,NaN,05-06-2024,BECADO DR. DIEGO IGNACIO MARÍN GARAY,13-03-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1892,Cáncer Colorectal,7950467,K,"GARAY MILLIE, GERARDO ALEJANDRO",Seguimiento,Pendiente Examen,26/06/2024 ????,DR ROJAS,NaN,"14/05/2024 en control medico solicita COLONO,...",NaN,NaN,NaN,DR. JULIO IGNACIO ROJAS ASTORGA,26-09-2023,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...,NaN,NaN
1263,Cáncer Colorectal . {decreto n° 4/2013},6287440,6,"SOTO CHÁVEZ, RICARDO BONIFACIO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#ND/#ND/-
1292,Cáncer Colorectal . {decreto n° 4/2013},6384691,0,"DE MONTI PIZARRO, ANGEL PATRICIO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22989355 / / 44042321 / - / -/./FIJO:...
1634,Cáncer Colorectal . {decreto n° 4/2013},8006466,7,"BRICEÑO BRAVO, MANUEL AMÉRICO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 7758528 / / 52284527 / 27758528 / -/5...


In [ ]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,15701292,2024-12-03 11:11:00,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
1,7629228,NaT,NaN
2,7029609,2024-09-01 14:39:00,Médico DIEGO RICARDO GARCÍA PRADO Control - Co...
3,8089466,2024-07-05 10:34:00,Médico KATYA MARINA CARRILLO GONZÁLEZ TC de Ab...
4,7844300,2024-03-13 12:51:00,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
...,...,...,...
1892,7950467,2024-05-14 10:29:00,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
1893,6287440,NaT,NaN
1894,6384691,NaT,NaN
1895,8006466,NaT,NaN


In [ ]:
# Convertir la columna de fecha a formato datetime
dataframe_final_2['Fecha Ultima atencion'] = pd.to_datetime(dataframe_final_2['Fecha Ultima atencion'])

# Formatear la columna de fecha en el formato deseado
dataframe_final_2['Fecha Ultima atencion'] = dataframe_final_2['Fecha Ultima atencion'].dt.strftime('%Y-%m-%d')


In [ ]:
dataframe_final_2 = dataframe_final_2.dropna()

In [ ]:
# Convertir la columna de fecha a formato datetime
dataframe_final_2['Fecha Ultima atencion'] = pd.to_datetime(dataframe_final_2['Fecha Ultima atencion'])

# Formatear la columna de fecha en el formato deseado
dataframe_final_2['Fecha Ultima atencion'] = dataframe_final_2['Fecha Ultima atencion'].dt.strftime('%d-%m-%Y')


In [ ]:
dataframe_final_2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,15701292,03-12-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
2,7029609,01-09-2024,Médico DIEGO RICARDO GARCÍA PRADO Control - Co...
3,8089466,05-07-2024,Médico KATYA MARINA CARRILLO GONZÁLEZ TC de Ab...
4,7844300,13-03-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
5,6165959,08-09-2023,Médico JOSE ANTONIO HERNANDEZ MONTALVO Derivac...
...,...,...,...
1887,6730627,15-12-2023,Médico DIEGO EDWARDS SILVA Rx Rodilla Izq F-L ...
1888,6787461,03-10-2024,Médico CRISTINA ANDREA RIVERA ARCOS Ecografia ...
1889,7107748,19-04-2024,Médico MANUEL ALEJANDRO PEÑALOZA HERRERA TC de...
1891,9864638,05-07-2024,Médico Zeyuan Zhang AngioTAC de Abdomen Image...


In [ ]:
import pandas
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(dataframe_final_2, index= False, header = True):
    ws.append(r)

for cell in ws['B'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_ultima_atencion_colorectal.xlsx'
wb.save(os.path.join(dir_actual, '..','CA COLORECTAL',f'{nombre_archivo}'))

## Guardar dataframe ultima atención 

In [ ]:
#nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_ultima_atencion_colorectal.xlsx'  # Nombre del archivo de Excel
#dataframe_final_2.to_excel(os.path.join(dir_actual, '..','CA COLORECTAL',f'{nombre_archivo}'), index=False)  # index=False para no incluir el índice en el archivo de Excel 


In [ ]:
df1 = df_concatenated_2.copy()
df2 = dataframe_final_2.copy()

df1['RUT'] = df1['RUT'].astype(str)
df2['RUT'] = df2['RUT'].astype(str)


# Realizar la fusión utilizando la columna 'RUT'
df_merged = pd.merge(df1, df2, on='RUT', how='left')



In [ ]:
df2

,RUT,Fecha Ultima atencion,Profesional Ultima atencion
0,15701292,03-12-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
2,7029609,01-09-2024,Médico DIEGO RICARDO GARCÍA PRADO Control - Co...
3,8089466,05-07-2024,Médico KATYA MARINA CARRILLO GONZÁLEZ TC de Ab...
4,7844300,13-03-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
5,6165959,08-09-2023,Médico JOSE ANTONIO HERNANDEZ MONTALVO Derivac...
...,...,...,...
1887,6730627,15-12-2023,Médico DIEGO EDWARDS SILVA Rx Rodilla Izq F-L ...
1888,6787461,03-10-2024,Médico CRISTINA ANDREA RIVERA ARCOS Ecografia ...
1889,7107748,19-04-2024,Médico MANUEL ALEJANDRO PEÑALOZA HERRERA TC de...
1891,9864638,05-07-2024,Médico Zeyuan Zhang AngioTAC de Abdomen Image...


In [ ]:
len(df1['RUT'].unique())

1896

In [ ]:
len(df2['RUT'].unique())

1280

In [ ]:
# Renombrar columnas para evitar colisiones
df_merged.rename(columns={'Fecha Ultima atencion_y': 'Fecha Ultima atencion', 'Profesional Ultima atencion_y': 'Profesional Ultima atencion'}, inplace=True)

# Eliminar columnas duplicadas
df_merged.drop(['Fecha Ultima atencion_x', 'Profesional Ultima atencion_x'], axis=1, inplace=True)



In [ ]:
df_merged[df_merged.duplicated(subset=['RUT'], keep=False)]

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,DIAGNOSTICOS,Telefono Consolidado2,Fecha Ultima atencion,Profesional Ultima atencion
758,Cancer Colorectal,5742623,3,GEORGINA AURORA CATALAN SAN MARTIN,Seguimiento,Pendiente Examen,20/05/2024????,Dra Carrillo,NaN,15/05/2024 EN CONTROL 07/05/2024 DRA SOLICITA ...,NaN,NaN,NaN,NaN,NaN,NaN,02-12-2024,Médico DIEGO IGNACIO MARÍN GARAY Control postq...
759,Cancer Colorectal,5742623,3,GEORGINA AURORA CATALAN SAN MARTIN,Seguimiento,Pendiente Examen,20/05/2024????,Dra Carrillo,NaN,15/05/2024 EN CONTROL 07/05/2024 DRA SOLICITA ...,NaN,NaN,NaN,NaN,NaN,NaN,02-12-2024,Médico DIEGO IGNACIO MARÍN GARAY Control postq...
1033,Cáncer Colorectal,5742623,3,"CATALÁN SAN MARTÍN, GEORGINA AURORA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02-12-2024,Médico DIEGO IGNACIO MARÍN GARAY Control postq...
1034,Cáncer Colorectal,5742623,3,"CATALÁN SAN MARTÍN, GEORGINA AURORA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02-12-2024,Médico DIEGO IGNACIO MARÍN GARAY Control postq...


In [ ]:
df_merged = df_merged.drop_duplicates(subset=['RUT'], keep='first')


In [ ]:
df_merged

,Problema Oncologico,RUT,DV,Nombre del Paciente,Etapa,Estado,Fecha Próximo Hito,Medico Responsable,Fecha de Comité,Observacion,Diagnosticos,Telefonos,CITAS FUTURAS,MEDICO,DIAGNOSTICOS,Telefono Consolidado2,Fecha Ultima atencion,Profesional Ultima atencion
0,Cáncer Colorectal,15701292,4,"TORRES MACHUCA, JOSSELLIN TIARE",Seguimiento,Pendiente Examen,2024-06-04 00:00:00,Dr Rojas,NaN,16/05/2024 en ultimo control 07/05/2024 indica...,NaN,NaN,04-06-2024,DR. JULIO IGNACIO ROJAS ASTORGA,NaN,NaN,03-12-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
1,Cáncer Colorectal,7629228,0,"AEDO MUÑOZ, MARÍA JEANNETTE",Ingreso,Agendado Médico,04-06-2024,Dr Tica,NaN,NaN,NaN,- / - / / / 30759801 / - / -/.,04-06-2024,Dr. ANIBAL TICA PLAYA,NaN,NaN,NaN,NaN
2,Cáncer Colorectal,7029609,8,"ALIAGA NAVARRO, CARLOS VALENTÍN",Seguimiento,Agendado Médico,04-06-2024,Dr Garcia,NaN,16/05/2024 en ultimo control 09/01/2024 indica...,NaN,NaN,04-06-2024,DR. JOSÉ ANTONIO HERNÁNDEZ MONTALVO,NaN,NaN,01-09-2024,Médico DIEGO RICARDO GARCÍA PRADO Control - Co...
3,Cáncer Colorectal,8089466,K,PATRICIO PARIS RODRÍGUEZ,NaN,NaN,04-06-2024,Dra Carrillo,NaN,"16/05/2024 CASO ESPCIAL, DADO DE ALTA POR Dra ...",NaN,977394489/991740981/978362791,04-06-2024,DR. JULIO IGNACIO ROJAS ASTORGA,NaN,NaN,05-07-2024,Médico KATYA MARINA CARRILLO GONZÁLEZ TC de Ab...
4,Cáncer Colorectal,7844300,6,"HIDALGO MADRIAGA, JEREMÍAS DEL ROSARIO",Seguimiento,Pendiente Examen,05-06-2024,Dr Marin,NaN,16/05/2024 EN ULTIMO CONTROL DR SOLCIITA COLON...,NaN,NaN,05-06-2024,BECADO DR. DIEGO IGNACIO MARÍN GARAY,NaN,NaN,13-03-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1894,Cáncer Colorectal,7950467,K,"GARAY MILLIE, GERARDO ALEJANDRO",Seguimiento,Pendiente Examen,26/06/2024 ????,DR ROJAS,NaN,"14/05/2024 en control medico solicita COLONO,...",NaN,NaN,NaN,DR. JULIO IGNACIO ROJAS ASTORGA,NaN,NaN,14-05-2024,Médico JULIO IGNACIO ROJAS ASTORGA Control - C...
1895,Cáncer Colorectal . {decreto n° 4/2013},6287440,6,"SOTO CHÁVEZ, RICARDO BONIFACIO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#ND/#ND/-,NaN,NaN
1896,Cáncer Colorectal . {decreto n° 4/2013},6384691,0,"DE MONTI PIZARRO, ANGEL PATRICIO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 22989355 / / 44042321 / - / -/./FIJO:...,NaN,NaN
1897,Cáncer Colorectal . {decreto n° 4/2013},8006466,7,"BRICEÑO BRAVO, MANUEL AMÉRICO",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,- / - / 7758528 / / 52284527 / 27758528 / -/5...,NaN,NaN


# Crear Citas futuras

In [ ]:
dir_actual = os.getcwd()

# Construir la ruta relativa al directorio donde se encuentra el archivo CSV
dir_archivo_csv = os.path.join(dir_actual, '..','..')

# Entrar a la carpeta "Datos HLF"
dir_datos_hlf = os.path.join(dir_archivo_csv, 'Datos HLF')


# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)

# Formatear mes con ceros a la izquierda si es necesario
month_formatted = "{:02d}".format(today.month)




ruta_del_archivo = os.path.join(ruta_reporteria,'Reportes_Ambulatorio','AgendasFuturas', f'{today.year}{month_formatted}{day_formatted}_Citas_Futuras.csv')


# Carga el archivo XLSX en un DataFrame
dataframe_fu = cargar_dataframe_ambulatorio(ruta_del_archivo)

In [ ]:
# Mover los nombres de las columnas una posición hacia la izquierda
new_columns = list(dataframe_fu.columns)
new_columns = new_columns[1:] + [new_columns[0]]
dataframe_fu.columns = new_columns


In [ ]:

dataframe_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1715889412635310311,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco de partes blandas cadera der,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715356221635300908,Enfermera/TENS AD Oriente,Hospitalización Domiciliaria MQ,21-05-2024,800,Agendada,AD Profesional / Tens Básica,CEX,N,N,NaN,...,988035938,979262107,NaN,NaN,El Tamarindo,La Florida,988035938,979262107,NaN,NaN
1715889364576310298,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco partes blandas cadera izq,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889364576310298,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco partes blandas cadera izq,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889060263310209,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Ecografia Cerebral en RN,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,VIC MACKENNA,La Florida,987551087,986127125,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715270935281300910,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1100,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,921712085,NaN,NaN,NaN,juan bagynka,La Florida,921712085,NaN,NaN,NaN
1713451086016300967,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1000,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,922269900,922269900,NaN,NaN,LAS ACACIAS,La Florida,922269900,922269900,NaN,NaN
1713451086016300967,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1000,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,922269900,922269900,NaN,NaN,LAS ACACIAS,La Florida,922269900,922269900,NaN,NaN
1715610771664310021,DR. LEONARDO JOSÉ SIRI ZUNINO,Infectología Adulto,19-12-2024,1140,Agendada,Control - Consulta integral de especialidades,CEX,N,N,NaN,...,0937564768,NaN,NaN,NaN,serrano,La Florida,0937564768,NaN,NaN,NaN


In [ ]:
# Eliminar duplicados por índice y mantener el primero
df_sin_duplicados = dataframe_fu.drop_duplicates(keep='first')


In [ ]:
df_sin_duplicados['ESTADO_CITA']

1715889412635310311    Agendada
1715356221635300908    Agendada
1715889364576310298    Agendada
1715889060263310209    Agendada
1713554280650310362    Agendada
                         ...   
1714662664397310870    Agendada
1713453317365300554    Agendada
1715270935281300910    Agendada
1713451086016300967    Agendada
1715610771664310021    Agendada
Name: ESTADO_CITA, Length: 28125, dtype: object

In [ ]:
df_sin_duplicados['ESTADO_CITA'].unique()

array(['Agendada', 'Anulada', 'En buzón de reasignación'], dtype=object)

In [ ]:
# Filtrar por las especialidades de Endocrinología Adulto y Endocrinología Infantil
endocrinologia_df_fu = df_sin_duplicados[df_sin_duplicados['ESPECIALIDAD_AGENDA'].isin(['Cirugía Proctológica','Imagenología AMB'])]
    # Filtrar las filas que no contienen 'No realizada' o 'Anulada' en la columna 'ESTADO_CITA'
endocrinologia_df_fu = endocrinologia_df_fu[~endocrinologia_df_fu['ESTADO_CITA'].isin(['Anulada', 'En buzón de reasignación'])]

In [ ]:
endocrinologia_df_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1715889412635310311,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco de partes blandas cadera der,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889364576310298,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco partes blandas cadera izq,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889060263310209,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Ecografia Cerebral en RN,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,VIC MACKENNA,La Florida,987551087,986127125,NaN,NaN
1716047834547300440,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,TC de Abdomen y Pelvis con MC,CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,...,56962841903,56994512452,NaN,NaN,rucalin,La Florida,56962841903,56994512452,NaN,NaN
1716047858150300443,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,"TAC de Tórax, sin MC",CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,...,56962841903,56994512452,NaN,NaN,rucalin,La Florida,56962841903,56994512452,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715704743898300503,Sala I-4,Imagenología AMB,29-07-2024,1100,Agendada,Rx Antebrazo Izq F-L,CEX,N,N,NaN,...,0940757310,22938680,NaN,NaN,Pje 4,La Florida,0940757310,22938680,NaN,NaN
1715719328671300942,Sala I-4,Imagenología AMB,30-07-2024,1000,Agendada,Rx Pelvis AP,CEX,N,N,NaN,...,56963132377,979506748,995202400,NaN,Los Papiros,La Florida,56963132377,979506748,995202400,NaN
1712323640596910122,Sala I-4,Imagenología AMB,04-10-2024,840,Agendada,Rx Cadera Der Crosstable,CEX,N,N,NaN,...,967396722,965981456,09965981456,NaN,SAN JOSE DE LA ESTRELLA,La Florida,967396722,965981456,09965981456,NaN
1712323464883910069,Sala I-4,Imagenología AMB,04-10-2024,830,Agendada,Rx Pelvis AP para evaluación de Prótesis,CEX,N,N,NaN,...,967396722,965981456,09965981456,NaN,SAN JOSE DE LA ESTRELLA,La Florida,967396722,965981456,09965981456,NaN


In [ ]:
# Convertir la columna de fecha a formato datetime
#endocrinologia_df_fu['CITAFECH'] = pd.to_datetime(endocrinologia_df_fu['CITAFECH'])

# Formatear la columna de fecha en el formato deseado
#endocrinologia_df_fu['CITAFECH'] = endocrinologia_df_fu['CITAFECH'].dt.strftime('%Y-%m-%d')

In [ ]:
endocrinologia_df_fu

,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,...,MOVIL_HABITUAL,TELEFONO1_HABITUAL,TELEFONO2_HABITUAL,TELEFONO3_HABITUALDIRECCION_HABITUAL,COMUNA_HABITUAL,MOVIL_CONTACTO,TELEFONO1_CONTACTO,TELEFONO2_CONTACTO,TELEFONO3_CONTACTO,CITA_KEY
1715889412635310311,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco de partes blandas cadera der,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889364576310298,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco partes blandas cadera izq,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,NaN,La Florida,987551087,986127125,NaN,NaN
1715889060263310209,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Ecografia Cerebral en RN,CEX,N,N,NaN,...,987551087,986127125,NaN,NaN,VIC MACKENNA,La Florida,987551087,986127125,NaN,NaN
1716047834547300440,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,TC de Abdomen y Pelvis con MC,CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,...,56962841903,56994512452,NaN,NaN,rucalin,La Florida,56962841903,56994512452,NaN,NaN
1716047858150300443,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,"TAC de Tórax, sin MC",CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,...,56962841903,56994512452,NaN,NaN,rucalin,La Florida,56962841903,56994512452,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715704743898300503,Sala I-4,Imagenología AMB,29-07-2024,1100,Agendada,Rx Antebrazo Izq F-L,CEX,N,N,NaN,...,0940757310,22938680,NaN,NaN,Pje 4,La Florida,0940757310,22938680,NaN,NaN
1715719328671300942,Sala I-4,Imagenología AMB,30-07-2024,1000,Agendada,Rx Pelvis AP,CEX,N,N,NaN,...,56963132377,979506748,995202400,NaN,Los Papiros,La Florida,56963132377,979506748,995202400,NaN
1712323640596910122,Sala I-4,Imagenología AMB,04-10-2024,840,Agendada,Rx Cadera Der Crosstable,CEX,N,N,NaN,...,967396722,965981456,09965981456,NaN,SAN JOSE DE LA ESTRELLA,La Florida,967396722,965981456,09965981456,NaN
1712323464883910069,Sala I-4,Imagenología AMB,04-10-2024,830,Agendada,Rx Pelvis AP para evaluación de Prótesis,CEX,N,N,NaN,...,967396722,965981456,09965981456,NaN,SAN JOSE DE LA ESTRELLA,La Florida,967396722,965981456,09965981456,NaN


In [ ]:
endocrinologia_df_fu['DV'] = endocrinologia_df_fu['DOCUMENTO'].str[-1]

endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['DOCUMENTO'].str[:-1]

In [ ]:
endocrinologia_df_fu = endocrinologia_df_fu[endocrinologia_df_fu['NOMBRE_TIPO_DOCUMENTO']=='RUN']

In [ ]:
endocrinologia_df_fu['RUT'] = endocrinologia_df_fu['RUT'].astype(int)

In [ ]:
endocrinologia_df_fu = endocrinologia_df_fu[['RUT','NOMBRE_AGENDA','ESPECIALIDAD_AGENDA','CITAFECH','CITAHORA','ESTADO_CITA','DESCRIPCION','ORIGEN','GES_CITA','GES_SOL','PATOLOGIA_GES_SOL','PACIENTE','CITA_CODUTEN','DOCUMENTO']]

In [ ]:
endocrinologia_df_fu

,RUT,NOMBRE_AGENDA,ESPECIALIDAD_AGENDA,CITAFECH,CITAHORA,ESTADO_CITA,DESCRIPCION,ORIGEN,GES_CITA,GES_SOL,PATOLOGIA_GES_SOL,PACIENTE,CITA_CODUTEN,DOCUMENTO
1715889412635310311,28368794,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco de partes blandas cadera der,CEX,N,N,NaN,ALANA ESTEFANÍA GARCIA BERMUDEZ,AC00542379,283687945
1715889364576310298,28368794,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Eco partes blandas cadera izq,CEX,N,N,NaN,ALANA ESTEFANÍA GARCIA BERMUDEZ,AC00542379,283687945
1715889060263310209,28368766,ECO I2-3,Imagenología AMB,21-05-2024,1030,Agendada,Ecografia Cerebral en RN,CEX,N,N,NaN,MAXIMILIANO ESTEBAN GARCIA BERMUDEZ,AC00542380,28368766K
1716047834547300440,7339477,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,TC de Abdomen y Pelvis con MC,CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,MERCEDES ROSA SEPÚLVEDA ARAVENA,AC00458224,73394775
1716047858150300443,7339477,TAC 2,Imagenología AMB,21-05-2024,900,Agendada,"TAC de Tórax, sin MC",CEX,S,S,CÁNCER COLORECTAL EN PERSONAS DE 15 AÑOS Y MÁS,MERCEDES ROSA SEPÚLVEDA ARAVENA,AC00458224,73394775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1715704743898300503,26604801,Sala I-4,Imagenología AMB,29-07-2024,1100,Agendada,Rx Antebrazo Izq F-L,CEX,N,N,NaN,FRANCISCO ISAAC TORRES CORNEJO,AC00390929,266048017
1715719328671300942,9006844,Sala I-4,Imagenología AMB,30-07-2024,1000,Agendada,Rx Pelvis AP,CEX,N,N,NaN,ROSA INÉS DE LAS MER ZÚÑIGA SOTO,AC00058536,9006844K
1712323640596910122,22889659,Sala I-4,Imagenología AMB,04-10-2024,840,Agendada,Rx Cadera Der Crosstable,CEX,N,N,NaN,MATÍAS ANTONIO HUMERES MADARIAGA,AC00387114,228896594
1712323464883910069,22889659,Sala I-4,Imagenología AMB,04-10-2024,830,Agendada,Rx Pelvis AP para evaluación de Prótesis,CEX,N,N,NaN,MATÍAS ANTONIO HUMERES MADARIAGA,AC00387114,228896594


In [ ]:
# Día actual
today = date.today()

# Fecha actual
now = datetime.now()

# Formatear día con ceros a la izquierda si es necesario
day_formatted = "{:02d}".format(today.day)


In [ ]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(endocrinologia_df_fu, index= False, header = True):
    ws.append(r)

for cell in ws['D'][1:]:
    cell.number_format = 'DD-MM-YYYY'
nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_citas_futuras_colorectal.xlsx'
wb.save(os.path.join(dir_actual, '..','CA COLORECTAL',f'{nombre_archivo}'))

In [ ]:
#nombre_archivo = f'{today.year}{month_formatted}{day_formatted}_citas_futuras_colorectal.xlsx'  # Nombre del archivo de Excel
#endocrinologia_df_fu.to_excel(os.path.join(dir_actual, '..','CA COLORECTAL',f'{nombre_archivo}'), index=False) 
